In [2]:
cl = ["A",
"B",
"G",
"R",
"Z",
"disparityL.x",
"disparityL.y",
"disparityR.x",
"disparityR.y",
"forward.left.u",
"forward.left.v",
"forward.right.u",
"forward.right.v",
"left.A",
"left.B",
"left.G"
"left.R"
"left.Z"
"whitebarmask.left.mask"
"whitebarmask.right.mask"]

In [9]:
def parse_channel(name):
    tags = name.split(".")
    viewlayer = tags[0] if len(tags)>1 else ""
    channel = tags[-1]
    return viewlayer, channel

class Layer:
    def __init__(self, name):
        self.name = ""
        self.channels = []

def group_layers(cl):
    patterns = ["ABGR", "BGR", "XYZ", "XY", "UVW", "UV"]
    layers = []
    i=0
    while i < len(cl):
        channel = cl[i]
        viewlayer, channel = parse_channel(channel)

        if len(layers)==0:
            layers.append(Layer(name=viewlayer))

        if layers[-1].name == viewlayer:
            layers[-1].channels.append(channel)
        else:
            layers.append(Layer(name=viewlayer))

        i+=1

    return layers

layers = group_layers(cl)
for layer in layers:
    print(layer.name, layer.channels)

 ['A', 'B', 'G', 'R', 'Z']
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []
